In [152]:
import torch
from PIL import Image, ImageDraw
from typing import Tuple
import numpy as np
import time
import cv2
import pandas as pd
import easydict
from pathlib import Path
import os


## validation 합치기

### validation 알집 풀기

In [2]:
from pathlib import Path
path = Path('C:/Users/User/document/final_project/preprocessing/v_data')

In [4]:
import zipfile
from pathlib import Path
from glob import glob
path = Path('C:/Users/User/document/final_project/preprocessing/v_data')
def unzip(path):
    path_list = path.rglob('*.zip')
    for paths in (path_list):
        with zipfile.ZipFile(paths, 'r') as zip_ref:
            zip_ref.extractall(str(paths)[:-4])
            # os.remove(zip_ref)
# unzip(path)

In [140]:
import json
def to_dataframe(path):
    path_list = path.glob('*/*')
    df_columns =[]

    for paths in path_list:
        json_path = paths.glob('*.json')
        for j in json_path:  # json_path 경로안에 있는 j 하나씩 읽음 
            with open(j, encoding = 'utf-8') as json_file:
                data = json.load(json_file)['images'][0]
                df_columns.append(data)
                break
    df_vaild = pd.DataFrame(df_columns)
    return df_vaild
v_df = to_dataframe(path) # validation 데이터 

In [141]:
v_df.head()

,file_name,width,height,imgfile,drug_N,...,mark_code_front_img,mark_code_back_img,mark_code_front,mark_code_back,change_date
0,K-044727_0_2_0_0_75_000_200.png,976,1280,K-044727_0_2_0_0_75_000_200.png,K-044727,...,,,,,None
1,K-044732_0_2_0_0_70_000_200.png,976,1280,K-044732_0_2_0_0_70_000_200.png,K-044732,...,,,,,20190222
2,K-044736_0_0_0_0_70_000_200.png,976,1280,K-044736_0_0_0_0_70_000_200.png,K-044736,...,,,,,20210609
3,K-044746_0_2_0_0_70_000_200.png,976,1280,K-044746_0_2_0_0_70_000_200.png,K-044746,...,,,,,20190114
4,K-044754_0_0_0_0_60_000_200.png,976,1280,K-044754_0_0_0_0_60_000_200.png,K-044754,...,,,,,20201221


### train 데이터 load

In [142]:
%cd C:\Users\User\document\final_project\preprocessing

C:\Users\User\document\final_project\preprocessing


In [143]:
data = pd.read_csv('data/df_merged.csv') # 원래 train 데이터 
data.head()

,idx,file_name,width,height,imgfile,...,form_code_name,change_date,area,iscrowd,bbox
0,0,K-019461_0_0_0_0_60_000_200.png,976,1280,K-019461_0_0_0_0_60_000_200.png,...,나정,20200930.0,50838,0,"[408, 519, 229, 222]"
1,1,K-019469_0_0_0_0_60_000_200.png,976,1280,K-019469_0_0_0_0_60_000_200.png,...,필름코팅정,20130129.0,72657,0,"[350, 490, 299, 243]"
2,2,K-019553_0_0_0_0_60_000_200.png,976,1280,K-019553_0_0_0_0_60_000_200.png,...,"서방성캡슐제, 펠렛",20201116.0,126169,0,"[249, 406, 449, 281]"
3,3,K-019699_0_0_0_0_60_000_200.png,976,1280,K-019699_0_0_0_0_60_000_200.png,...,필름코팅정,20190225.0,54488,0,"[357, 541, 278, 196]"
4,4,K-019700_0_0_0_0_60_000_200.png,976,1280,K-019700_0_0_0_0_60_000_200.png,...,필름코팅정,20190225.0,94166,0,"[325, 523, 394, 239]"


### preprocessing and load

In [144]:
def col_delete(df):
    df.drop(['drug_S', 'width', 'height', 'mark_code_front', 'mark_code_back', 'line_front','line_back', 'mark_code_front_anal', 'mark_code_back_anal', 'mark_code_front_img', 'mark_code_back_img',
                  'color_class2', 'file_name', 'dl_company_en', 'di_company_mf', 'di_company_mf_en', 'img_regist_ts',
                  'change_date', 'back_color', 'light_color', 'camera_la', 'camera_lo', 'print_back'],
                  axis = 1, inplace = True)
    df['leng_long'].fillna(0, inplace = True)
    df['leng_short'].fillna(0, inplace = True)
    df['thick'].fillna(0, inplace = True)
    df.fillna('unknown', inplace = True)
    
col_delete(v_df)

In [145]:
def col_delete2(data):
    data.drop(['idx','drug_S', 'width', 'height',  'file_name',  'img_regist_ts','change_date', 'back_color', 'light_color', 'camera_la', 'camera_lo', 'print_back'],axis = 1, inplace = True)
    data['leng_long'].fillna(0, inplace = True)
    data['leng_short'].fillna(0, inplace = True)
    data['thick'].fillna(0, inplace = True)
    data.fillna('unknown', inplace = True)
col_delete2(data)

In [146]:
result_df = pd.concat([data, v_df])
result_df =result_df[['drug_N','dl_name','dl_name_en','dl_material','dl_material_en','item_seq','di_class_no','di_edi_code','form_code_name','bbox']]
result_df =result_df.reset_index(drop=True)
result_df.head()
# result_df.columns

,drug_N,dl_name,dl_name_en,dl_material,dl_material_en,item_seq,di_class_no,di_edi_code,form_code_name,bbox
0,K-019461,미래트리메부틴정 100mg/병,Trimebutine Tab. Mirae,트리메부틴말레산염,Trimebutine,200607006,[02390]기타의 소화기관용약,657500470,나정,"[408, 519, 229, 222]"
1,K-019469,큐레틴정(빌베리건조엑스),Qretin Tab.,빌베리건조엑스,Bilberry Dried Ext.,200607018,[01310]안과용제,"650101140,A28302561",필름코팅정,"[350, 490, 299, 243]"
2,K-019553,콘택골드캡슐 10mg/PTP,Contac Gold Cap.,클로르페니라민말레산염|벨라돈나총알칼로이드|페닐레프린염산염,Phenylephrine Hydrochloride| Belladonna Total ...,200607735,[01410]항히스타민제,642102330,"서방성캡슐제, 펠렛","[249, 406, 449, 281]"
3,K-019699,에스케이코스카플러스정,Cosca Plus Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,200608151,[02140]혈압강하제,"644703770,653802290,A20753971",필름코팅정,"[357, 541, 278, 196]"
4,K-019700,에스케이코스카플러스에프정,Cosca Plus-F Tab. SK,로사르탄칼륨|히드로클로로티아지드,Hydrochlorothiazide| Losartan potassium,200608152,[02140]혈압강하제,"644703780,653802280,A20753981",필름코팅정,"[325, 523, 394, 239]"


In [147]:
df_information = pd.read_csv('data/e약은요.csv') # 약 내용 데이터 
df_information.drop(['제품명','공개일자','수정일자','업체명','이 약은 어떻게 보관해야 합니까?','이 약을 사용하기 전에 반드시 알아야 할 내용은 무엇입니가?'],axis=1,inplace=True)

In [319]:
merge_df_tv = pd.merge(result_df,df_information, how='left', left_on='item_seq', right_on='품목일련번호')
keys = ['이 약의 효능은 무엇입니까?','이 약은 어떻게 사용합니까?','이 약을 사용하기 전에 반드시 알아야 할 내용은 무엇입니가?','이 약의 사용상 주의사항은 무엇입니까?'
        ,'이 약을 사용하는 동안 주의해야 할 약 또는 음식은 무엇입니까?','이 약은 어떤 이상반응이 나타날 수 있습니까?']
values = ['효능','사용 방법','알아야 할 내용','주의사항','주의음식','부작용','성분명(영어)','제조사명','제조사명(영어)','약품 분류','품목기준 코드','전문의약품 여부','EDI코드','제형',
'모양','식별문자(앞)','식별문자(뒤)','색상','정제 분류명','영역']
my_dict = dict(zip(keys, values))
merge_df_tv = merge_df_tv.rename(columns=my_dict)
merge_df_tv.to_csv('merge_df_tv.csv', index=False,encoding='utf-8-sig')

In [328]:
merge_df_tv['di_edi_code']

0                           657500470
1                 650101140,A28302561
2                           642102330
3       644703770,653802290,A20753971
4       644703780,653802280,A20753981
                    ...              
4519                        648507790
4520                        055100300
4521                        665507030
4522                        657203390
4523                        622803300
Name: di_edi_code, Length: 4524, dtype: object

## 결과랑 연결

In [150]:
%cd C:/Users/User/document/final_project/drugweb/flask/final_yolo_5

C:\Users\User\document\final_project\drugweb\flask\final_yolo_5


In [151]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', _verbose=False)

Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-7-18 Python-3.10.11 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 8358625 parameters, 0 gradients, 20.0 GFLOPs
Adding AutoShape... 


In [153]:
base_path = Path(os.getcwd())
base_list = base_path/'a.png'
base_list

WindowsPath('C:/Users/User/document/final_project/drugweb/flask/final_yolo_5/a.png')

In [238]:
# df_warming = pd.read_csv('df_warming.csv') # 주의 데이터
# df_warming['code조합'] = df_warming['제품코드A'].astype(str) + ',' + df_warming['제품코드B'].astype(str)
# df_warming['code조합'] = df_warming['code조합'].str.split(',')
# df_warming.to_csv('df_warming.csv', index=False,encoding='utf-8-sig') # 주의 code list 만들어서 저장
df_warming['code조합'].iloc[1]

['648101510', '647804481']

In [208]:

from PIL import Image, ImageDraw
from typing import Tuple
import numpy as np
import time
import cv2
import pandas as pd
import easydict
from pathlib import Path
import os

base_path = Path(os.getcwd())
image_path = base_path/'a.png'
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')
img = cv2.imread(str(image_path))
results = model(img)
df = results.pandas().xyxy[0] # 결과 데이터
merge_df_tv = pd.read_csv('merge_df_tv.csv') # 4000개 데이터
image_df = pd.merge(df,merge_df_tv, how='left', left_on='name', right_on='drug_N')
print(image_df[['dl_name','dl_material','confidence','품목일련번호', '주성분', '효능','사용 방법', '주의사항', '주의음식', '부작용','di_edi_code']])
# 문제가 생겼을경우만 출력 
df_warming = pd.read_csv('df_warming.csv') # 주의 데이터 
for i,v in enumerate(df_warming['code조합']):
    if set(v).issubset(set(image_df['di_edi_code'].values))==True:
        print('문제 조합 : ' ,v ) # 문제 조합 
        print(image_df[image_df['di_edi_code']==v[0]]['name'].values) # 문제조합 코드 이름 
        print(image_df[image_df['di_edi_code']==v[1]]['name'].values) # 문제조합 코드 이름
        print('주의사항 : ' + df_warming.iloc[i]["상세정보"] + '이 생길수 있습니다') # 문제 조합 상세정보 


Using cache found in C:\Users\User/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-7-18 Python-3.10.11 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 8358625 parameters, 0 gradients, 20.0 GFLOPs
Adding AutoShape... 


           dl_name                                        dl_material   
0      히트펜정 25mg/병                             에페드린염산염|카페인무수물|아세트아미노펜  \
1  에스슬림정 556.7mg/병  당귀|작약|천궁|연교|박하|방풍|마황|대황|망초|길경|황금|석고|생강|치자|형개|백...   
2            유앤디캡슐                         비페닐디메틸디카르복실레이트혼합물|우르소데옥시콜산   
3           헤파텍트캡슐                            비페닐디메틸디카르복실레이트|우르소데옥시콜산   

   confidence  품목일련번호  주성분  ... 사용 방법 주의사항 주의음식  부작용 di_edi_code  
0    0.755084     NaN  NaN  ...   NaN  NaN  NaN  NaN   690300140  
1    0.642998     NaN  NaN  ...   NaN  NaN  NaN  NaN     unknown  
2    0.435059     NaN  NaN  ...   NaN  NaN  NaN  NaN   694001530  
3    0.258761     NaN  NaN  ...   NaN  NaN  NaN  NaN   658502020  

[4 rows x 11 columns]


### test

In [249]:
values_list = image_df['di_edi_code'].tolist()
values_list = ['646002281','648101510','694001530','658502020','646002281']

In [342]:
for i,v in enumerate(df_warming['code조합']):
    if set(v).issubset(set(values_list)):
        print('문제 조합 코드 : ' , v)
        print(image_df[image_df['di_edi_code']==v[0]]['name'].values)
        print(image_df[image_df['di_edi_code']==v[1]]['name'].values)
        print('주의사항 : ' + df_warming.iloc[i]["상세정보"] + '증상이 생길수 있습니다')

문제 조합 코드 :  ['648101510', '646002281']
[]
[]
주의사항 : 중증의 위장관계 이상반응증상이 생길수 있습니다
문제 조합 코드 :  ['648101510', '646002281']
[]
[]
주의사항 : 중증의 위장관계 이상반응증상이 생길수 있습니다
문제 조합 코드 :  ['646002281', '648101510']
[]
[]
주의사항 : 중증의 위장관계 이상반응증상이 생길수 있습니다


In [352]:
merge_df_tv[merge_df_tv['di_edi_code']=='648101510']

,drug_N,dl_name,dl_name_en,dl_material,dl_material_en,...,효능,사용 방법,주의사항,주의음식,부작용


In [345]:
v[0] 

'656003820'

In [340]:
image_df[image_df['di_edi_code']==v[0]]['name'].values

array([], dtype=object)

In [329]:
image_df[image_df['di_edi_code']==v[0]].index

Index([], dtype='int64')

In [313]:
merge_df_tv['di_edi_code'].iloc[1]

'650101140,A28302561'

In [307]:
[index for index, value in merge_df_tv['di_edi_code'].items() if v[0] in value]

[]

In [301]:
merge_df_tv[merge_df_tv['di_edi_code'].values.contain(v[0])]
# index
# print(image_df.iloc[index]['dl_name'].values)

AttributeError: 'Series' object has no attribute 'contain'

In [288]:
image_df.columns

Index(['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class', 'name', 'drug_N',
       'dl_name', 'dl_name_en', 'dl_material', 'dl_material_en', 'item_seq',
       'di_class_no', 'di_edi_code', 'form_code_name', 'bbox', '품목일련번호', '주성분',
       '효능', '사용 방법', '주의사항', '주의음식', '부작용'],
      dtype='object')

In [248]:
print(set(v)) , print(set(values_list))

{'646002281', '648101510'}
{'641105840', '658502020', '654004590', '646002281', '694001530'}


(None, None)

In [242]:
for i,v in enumerate(df_warming['code조합']):
    print(type(v))
    break

<class 'list'>


In [224]:
for i,v in enumerate(df_warming['code조합']):
    if set(v).issubset(values_list)==True:
        print('문제 조합 : ' + str(v) )
        print('주의사항 : ' + df_warming.iloc[i]["상세정보"] + '증상이 생길수 있습니다')

In [223]:
df_warming.iloc[0]["상세정보"] 

'중증의 위장관계 이상반응'

In [221]:
a = ['648101510', '646002281']
set(a).issubset(values_list)

True

In [219]:
for i,v in enumerate(df_warming['code조합']):
    print(i,v)
    break

0 ['648101510', '646002281']


In [90]:
# lst =[]
# for i in data['file_name']:
#     i = i[0:8]
#     lst.append(i)
# data['name'] = lst